# Testing

This is a test notebook to try different parts of the scraper.

In [3]:
import pandas as pd
from requests_html import HTMLSession

In [108]:
# Getting tanscript from strip 1

s = HTMLSession()

base_url = 'https://www.explainxkcd.com/wiki/index.php'

page = 145
url = base_url+str(page)

r = s.get(url)

transcript = r.html.find('dl')


In [37]:
# r.html.xpath('//*[@id="Transcript"]/descendant::text()')

transcript_lines = r.html.xpath("//h2[contains(.,'Transcript')]//following-sibling::dl")

In [43]:
full_transcript = ''
for line in transcript_lines:
    print(line.text)
    full_transcript += line.text

[T-Rex, a large green Tyrannosaurus, holds out his small arms to each side and the tail pointing up while speaking with a wide open pink mouth showing all his teeth. All the text is written like on a typewriter with both caps and lowercase letters, which is not normal in xkcd.]
T-Rex: THINGS I AM UPPITY ABOUT: "They" as a third-person singular gender-free pronoun.
[Zoom in on T-Rex head holding his hands up under his mouth, and mouth even wider open so also the red tongue can be seen.]
T-Rex: I'm all for it!
[Zoom out to show T-Rex to the left, mouth almost closed, arms in normal position, the tail pointing up, and lifting his left leg ready to smash his foot down through the roof of a brown log cabin with chimney and porch with a blue car holding in front of the house to the right. Further right is a smaller white/yellow dinosaur, Dromiceiomimus, standing away from T-Rex, but turning its long neck toward him.]
Dromiceiomimus: But isn't that terrible grammar?
T-Rex: Only by recent conv

In [44]:
# Getting all text in a single string
full_transcript

'[T-Rex, a large green Tyrannosaurus, holds out his small arms to each side and the tail pointing up while speaking with a wide open pink mouth showing all his teeth. All the text is written like on a typewriter with both caps and lowercase letters, which is not normal in xkcd.]\nT-Rex: THINGS I AM UPPITY ABOUT: "They" as a third-person singular gender-free pronoun.[Zoom in on T-Rex head holding his hands up under his mouth, and mouth even wider open so also the red tongue can be seen.]\nT-Rex: I\'m all for it![Zoom out to show T-Rex to the left, mouth almost closed, arms in normal position, the tail pointing up, and lifting his left leg ready to smash his foot down through the roof of a brown log cabin with chimney and porch with a blue car holding in front of the house to the right. Further right is a smaller white/yellow dinosaur, Dromiceiomimus, standing away from T-Rex, but turning its long neck toward him.]\nDromiceiomimus: But isn\'t that terrible grammar?\nT-Rex: Only by recent

In [65]:
# Getting strip number and date
top = r.html.find('a.external.text')[0]


#mw-content-text > div > table > tbody > tr:nth-child(1) > td > ul > li.plainlinks > a > span

In [70]:
top.find('span')[0].text

'Comic\xa0#145\xa0(August\xa018,\xa02006)'

In [71]:
top.find('span')[0].text.replace(u'\xa0', ' ')

'Comic #145 (August 18, 2006)'

In [72]:
text = top.find('span')[0].text.replace(u'\xa0', ' ')

In [75]:
# Getting comic number:
# Everything after the # and before an epmty space
text[text.index("#") + 1 : text.index("(") - 1]

'145'

In [76]:
# Getting the top text with comic number and date
num_and_date = r.html.find('a.external.text')[0].find('span')[0].text

# Clean the text
num_and_date = num_and_date.replace(u'\xa0', ' ')

# So, the comic strip number is given by:
strip_number = num_and_date[num_and_date.index("#") + 1 : num_and_date.index("(") - 1]
print(strip_number)

145


In [79]:
# Getting the date:
strip_date = num_and_date[num_and_date.index("(") + 1 : num_and_date.index(")")]
print(strip_date)

August 18, 2006


In [97]:
## Getting the strip title
strip_title = r.html.xpath('//*[@id="mw-content-text"]/div/table')[0].find("td")[1].find('b')[0].text

#mw-content-text > div > table > tbody > tr:nth-child(2) > td > b

strip_title

'Parody Week: Dinosaur Comics'

In [130]:
# Putting it all together
s = HTMLSession()

base_url = 'https://www.explainxkcd.com/wiki/index.php/'

page = 2645
url = base_url+str(page)

r = s.get(url)

# Getting the transcript as single string
transcript_lines = r.html.xpath("//h2[contains(.,'Transcript')]//following-sibling::dl")

full_transcript = ''
for line in transcript_lines:
    full_transcript += line.text

# Getting the top text with comic number and date
num_and_date = r.html.find('a.external.text')[0].find('span')[0].text

# Clean the text
num_and_date = num_and_date.replace(u'\xa0', ' ')

# Comic strip number is given by:
strip_number = num_and_date[num_and_date.index("#") + 1 : num_and_date.index("(") - 1]

# Getting the date:
strip_date = num_and_date[num_and_date.index("(") + 1 : num_and_date.index(")")]

## Getting the strip title
strip_title = r.html.xpath('//*[@id="mw-content-text"]/div/table')[0].find("td")[1].find('b')[0].text

print(strip_number, strip_date, strip_title, full_transcript, sep=" | ")

2645 | July 13, 2022 | The Best Camera | [Each panel features an image of space, with text printed in white at the top of each panel. The first panel says:]
They say the best camera is the one you have with you.[A panel showing more stars and galaxies visible.]
It turns out[A panel showing even more stars and galaxies visible. At the center of the panel is an outline drawing in white of the James Webb Space Telescope.]
they're wrong.


In [132]:
# Going to next page
next_button = r.html.xpath("//span[contains(.,'Next')]//parent::a")

if next_button:
    next_url = next_button[0].attrs['href']
    print('https://www.explainxkcd.com' + next_url)
else:
    print('Last page!')

# #mw-content-text > div > table > tbody > tr:nth-child(1) > td > ul > li:nth-child(4) > a

Last page!


In [133]:
def get_next_page(url):

    r = s.get(url)

    next_button = r.html.xpath("//span[contains(.,'Next')]//parent::a")

    if next_button:
        next_url = next_button[0].attrs["href"]
        return "https://www.explainxkcd.com" + next_url

In [146]:
page = 421
url = base_url+str(page)

get_next_page(url)

'https://www.explainxkcd.com/wiki/index.php/422'

In [145]:
url = base_url+str(1)
while True:
    url = get_next_page(url)
    print(url)

https://www.explainxkcd.com/wiki/index.php/2
https://www.explainxkcd.com/wiki/index.php/3
https://www.explainxkcd.com/wiki/index.php/4
https://www.explainxkcd.com/wiki/index.php/5
https://www.explainxkcd.com/wiki/index.php/6
https://www.explainxkcd.com/wiki/index.php/7
https://www.explainxkcd.com/wiki/index.php/8
https://www.explainxkcd.com/wiki/index.php/9
https://www.explainxkcd.com/wiki/index.php/10
https://www.explainxkcd.com/wiki/index.php/11
https://www.explainxkcd.com/wiki/index.php/12
https://www.explainxkcd.com/wiki/index.php/13
https://www.explainxkcd.com/wiki/index.php/14
https://www.explainxkcd.com/wiki/index.php/15
https://www.explainxkcd.com/wiki/index.php/16
https://www.explainxkcd.com/wiki/index.php/17
https://www.explainxkcd.com/wiki/index.php/18
https://www.explainxkcd.com/wiki/index.php/19
https://www.explainxkcd.com/wiki/index.php/20
https://www.explainxkcd.com/wiki/index.php/21
https://www.explainxkcd.com/wiki/index.php/22
https://www.explainxkcd.com/wiki/index.php

KeyboardInterrupt: 